<a href="https://colab.research.google.com/github/MPGarg/-TSCAI/blob/main/Submission_EVA8_Session_2_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function #compatibility in Python versions 3.x and 2.x
import torch                          #provides a flexible N-dimensional array or Tensor, which supports basic routines for indexing, slicing, transposing
import torch.nn as nn                 #various neural network layer like regular fully-connected layers, convolutional layers, max pool etc
import torch.nn.functional as F       #provides various activation functions like Relu
import torch.optim as optim           #package implementing various optimization algorithms
from torchvision import datasets, transforms  #datasets for in-built databases and transforms for common image transformations like rotation, flip, crop etc

In [ ]:
class Net(nn.Module):                                     #class Net
    def __init__(self):                                   #constructor
        super(Net, self).__init__()                       #super class constructor
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)       #input=(28x28x1) OUtput=(28x28x32)  RF=(3x3)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)      #input=(28x28x32) OUtput=(28x28x64)  RF=(5x5)
        #torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        self.pool1 = nn.MaxPool2d(2, 2)                   #input=(28x28x64) OUtput=(14x14x64)  RF=(10x10) #assume MaxPooling doubles the RF for now
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     #input=(14x14x64) OUtput=(14x14x128)  RF=(12x12)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)    #input=(14x14x128) OUtput=(14x14x256)  RF=(14x14)
        self.pool2 = nn.MaxPool2d(2, 2)                   #input=(14x14x256) OUtput=(7x7x256)  RF=(28x28) #assume MaxPooling doubles the RF for now  
        self.conv5 = nn.Conv2d(256, 512, 3)  #padding 0   #input=(7x7x256) OUtput=(5x5x512)  RF=(30x30)
        self.conv6 = nn.Conv2d(512, 1024, 3) #padding 0   #input=(5x5x512) OUtput=(3x3x1024)  RF=(32x32)
        self.conv7 = nn.Conv2d(1024, 10, 3)  #padding 0   #input=(3x3x1024) OUtput=(1x1x10)  RF=(34x34)

    def forward(self, x):                                         #it's the forward function that defines the network structure
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #relu activation on conv1 with x as input 
                                                                  #-> passing output of this as input to conv2 with relu as activation 
                                                                  #-> applying max pooling to output of previous layer and storing back in x
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # passing x to conv3 with relu activation
                                                                  #-> passing output to conv4 with relu activation
                                                                  #-> applying max pooling to output of previous layer and storing back in x
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             #passing x to conv5 with relu activation
                                                                  #-> passing output to conv6 with relu activation and storing back in x
        x = F.relu(self.conv7(x))                                 #passing x as input to conv7 and storing data in x
        x = x.view(-1, 10)                                        #reshaping it with 10 columns
        return F.log_softmax(x)                                   #applying softmax as output function

In [ ]:
!pip install torchsummary                                         #install torchsummary
from torchsummary import summary                                  #import summary to show report
use_cuda = torch.cuda.is_available()                              #check gpu availability 
device = torch.device("cuda" if use_cuda else "cpu")              #use cuda is gpu available else cpu
model = Net().to(device)                                          #set device for computation and create model object with Net class
summary(model, input_size=(1, 28, 28))                            #print summary of mode with input as 28x28 grey scale image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 

<ipython-input-2-58daf3534683>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)                                   #applying softmax as output function


In [ ]:


torch.manual_seed(1)                      #Sets the seed for generating random numbers
batch_size = 128                          #batch size of images to be processed

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} #this is for optimizing speed if gpu in use
train_loader = torch.utils.data.DataLoader(                         #MNIST images data loading for train dataset
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([                  #clubbing of transformations
                        transforms.ToTensor(),                      #Convert a Image to tensor
                        transforms.Normalize((0.1307,), (0.3081,))  #mean and std deviation 
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                  #batch size with shuffle allowed
test_loader = torch.utils.data.DataLoader(                          #this is for test dataset
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm                                           #progress bar library                     
def train(model, device, train_loader, optimizer, epoch):       #defining train function
    model.train()                                               #training model
    pbar = tqdm(train_loader)                                   #progress bas object for train data
    for batch_idx, (data, target) in enumerate(pbar):           #get batch id with it's data & labels for training
        data, target = data.to(device), target.to(device)       #set data for training
        #For every mini-batch during the training phase, explicitly set the gradients to zero before starting to do backpropragation 
        #updating the Weights because PyTorch accumulates the gradients on subsequent backward passes
        optimizer.zero_grad()
        output = model(data)                                    #get output from trained model
        loss = F.nll_loss(output, target)                       #calculate loss between tarrget & output
        loss.backward()                                         #backward pass to adjust weights
        #makes the optimizer iterate over all parameters (tensors) it is supposed to update and use their internally stored grad to update their values
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')  #description of bar


def test(model, device, test_loader):                           #define test function
    model.eval()                                                #evaluate model
    test_loss = 0
    correct = 0
    #The wrapper with torch.no_grad() temporarily sets all of the requires_grad flags to false
    #it deactivates autograd engine thus performing inference without Gradient Calculation
    #To make sure there's no leak test data into the model
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)                                            #get prediction
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()           #correct prediction sum

    test_loss /= len(test_loader.dataset)                                   #average loss

    #print details
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)                                                    #create model
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)            #SGD optimizer with learning rate as 0.01 and momentum = 0.9

for epoch in range(1, 5):                                                   #number of epochs or number of times model is trained
    train(model, device, train_loader, optimizer, epoch)                    #training model with epoch times
    test(model, device, test_loader)                                        #test model

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-58daf3534683>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)                                   #applying softmax as output function
loss=1.1566210985183716 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.68it/s]



Test set: Average loss: 1.0126, Accuracy: 5761/10000 (58%)



loss=0.8221840262413025 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.87it/s]



Test set: Average loss: 0.7516, Accuracy: 6835/10000 (68%)



loss=0.7284088730812073 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.98it/s]



Test set: Average loss: 0.7455, Accuracy: 6844/10000 (68%)



loss=0.5795119404792786 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.17it/s]



Test set: Average loss: 0.7514, Accuracy: 6838/10000 (68%)

